In [ ]:
!nvcc --version
!nvidia-smi
import torch
torch.__version__

In [1]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git@5aeb252b194b93dc2879b4ac34bc51a31b5aee13'

  Cloning https://github.com/facebookresearch/detectron2.git (to revision 5aeb252b194b93dc2879b4ac34bc51a31b5aee13) to /tmp/pip-req-build-eqssuz_a
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-eqssuz_a
  Running command git rev-parse -q --verify 'sha^5aeb252b194b93dc2879b4ac34bc51a31b5aee13'
  Running command git fetch -q https://github.com/facebookresearch/detectron2.git 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
  Running command git checkout -q 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
  Resolved https://github.com/facebookresearch/detectron2.git to commit 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.8 MB/s eta 

In [11]:
%cd /content/
!git clone https://github.com/DomMcOyle/TACO-expl.git
%cd /content/TACO-expl
!git checkout maskdino
!git pull origin maskdino
%cd /content/TACO-expl/MaskDINO
!pip install -r requirements.txt
%cd /content/TACO-expl/MaskDINO/maskdino/modeling/pixel_decoder/ops
!sh make.sh

/content
Cloning into 'TACO-expl'...
remote: Enumerating objects: 2338, done.
remote: Counting objects: 100% (2338/2338), done.
remote: Compressing objects: 100% (1124/1124), done.
remote: Total 2338 (delta 1226), reused 2258 (delta 1175), pack-reused 0
Receiving objects: 100% (2338/2338), 130.95 MiB | 20.99 MiB/s, done.
Resolving deltas: 100% (1226/1226), done.
/content/TACO-expl
Branch 'maskdino' set up to track remote branch 'maskdino' from 'origin'.
Switched to a new branch 'maskdino'
From https://github.com/DomMcOyle/TACO-expl
 * branch            maskdino   -> FETCH_HEAD
Already up to date.
/content/TACO-expl/MaskDINO
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 2.8 MB/s eta 0:00:00
/content/TACO-expl/MaskDINO/maskdino/modeling/pixel_decoder/ops
running build
running build_py
creating build
creating build/lib.linux-x86_64-cpython-310
creating build/lib.linux-x86_64-cpython-310/functions
copying functions/ms_deform_attn_func.py -> build/lib.linux-x86_64-cpython-310/f

In [1]:
from google.colab import drive
drive.mount("/content/MyDrive/", force_remount = True)

Mounted at /content/MyDrive/


In [2]:
%cd /content/
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

/content


In [4]:
%cd /content/TACO-expl/
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

import os.path
import json
import argparse
import numpy as np
import random
import datetime as dt
import copy
import torch.nn as nn
from sklearn.model_selection import train_test_split
from pathlib import Path

import torch
import math
import torch.utils.data
from torch.utils.data import DataLoader
from pycocotools import mask as coco_mask

[Errno 2] No such file or directory: '/content/TACO-expl/'
/content


In [5]:
DEFAULT_RANDOM_SEED = 42
# basic random seed
def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
# torch random seed
def seedTorch(seed=DEFAULT_RANDOM_SEED):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
# combine
def seedEverything(seed=DEFAULT_RANDOM_SEED):
    seedBasic(seed)
    seedTorch(seed)

seedEverything()

In [7]:
keep_categories = ["Bottle", "Bottle cap", "Can", "Cigarette", "Cup",
                   "Lid", "Plastic bag & wrapper", "Pop tab", "Straw"]

def create_map(original, keep_supercategories):
  class_map = {}
  for cat in original:
    if cat["supercategory"] in keep_supercategories:
      class_map[cat["name"]] = cat["supercategory"]
    else:
      class_map[cat["name"]] = "Other"
  return class_map

def replace_dataset_classes(dataset, class_map):
      """ Replaces classes of dataset based on a dictionary"""
      class_new_names = list(set(class_map.values()))
      class_new_names.sort()
      class_originals = copy.deepcopy(dataset['categories'])
      dataset['categories'] = []
      class_ids_map = {}  # map from old id to new id

      # Assign background id 0
      has_background = False
      if 'Background' in class_new_names:
          if class_new_names.index('Background') != 0:
              class_new_names.remove('Background')
              class_new_names.insert(0, 'Background')
          has_background = True

      # Replace categories
      for id_new, class_new_name in enumerate(class_new_names):
          # Make sure id:0 is reserved for background
          id_rectified = id_new
          if not has_background:
              id_rectified += 1

          category = {
              'supercategory': '',
              'id': id_rectified,  # Background has id=0
              'name': class_new_name,
          }
          dataset['categories'].append(category)
          # Map class names
          for class_original in class_originals:
              if class_map[class_original['name']] == class_new_name:
                  class_ids_map[class_original['id']] = id_rectified

      # Update annotations category id tag
      for ann in dataset['annotations']:
          ann['category_id'] = class_ids_map[ann['category_id']]

In [12]:
with open("/content/TACO-expl/data/annotations.json", "r") as f: #"/content/TACO/data/annotations.json"
    dataset = json.loads(f.read())

class_map = create_map(dataset["categories"], keep_categories)
replace_dataset_classes(dataset, class_map)
dataset["categories"]

[{'supercategory': '', 'id': 1, 'name': 'Bottle'},
 {'supercategory': '', 'id': 2, 'name': 'Bottle cap'},
 {'supercategory': '', 'id': 3, 'name': 'Can'},
 {'supercategory': '', 'id': 4, 'name': 'Cigarette'},
 {'supercategory': '', 'id': 5, 'name': 'Cup'},
 {'supercategory': '', 'id': 6, 'name': 'Lid'},
 {'supercategory': '', 'id': 7, 'name': 'Other'},
 {'supercategory': '', 'id': 8, 'name': 'Plastic bag & wrapper'},
 {'supercategory': '', 'id': 9, 'name': 'Pop tab'},
 {'supercategory': '', 'id': 10, 'name': 'Straw'}]

In [13]:
classes = [elem["name"] for elem in dataset["categories"]]
print(classes)

['Bottle', 'Bottle cap', 'Can', 'Cigarette', 'Cup', 'Lid', 'Other', 'Plastic bag & wrapper', 'Pop tab', 'Straw']


In [10]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog

help(register_coco_instances)

Help on function register_coco_instances in module detectron2.data.datasets.coco:

register_coco_instances(name, metadata, json_file, image_root)
    Register a dataset in COCO's json annotation format for
    instance detection, instance segmentation and keypoint detection.
    (i.e., Type 1 and 2 in http://cocodataset.org/#format-data.
    `instances*.json` and `person_keypoints*.json` in the dataset).
    
    This is an example of how to register a new dataset.
    You can do something similar to this function, to register new datasets.
    
    Args:
        name (str): the name that identifies a dataset, e.g. "coco_2014_train".
        metadata (dict): extra metadata associated with this dataset.  You can
            leave it as an empty dict.
        json_file (str): path to the json instance annotation file.
        image_root (str or path-like): directory which contains all the images.



In [14]:
train_annotation_file = '/content/TACO-expl/data/annotations_off_0_train.json'
val_annotation_file = '/content/TACO-expl/data/annotations_off_0_val.json'
test_annotation_file = '/content/TACO-expl/data/annotations_off_0_test.json'


img_dir = '/content/MyDrive/MyDrive/official/'

register_coco_instances("TACO_train", {}, train_annotation_file, img_dir)
MetadataCatalog.get("TACO_train").set(thing_classes = classes)
dataset_dicts_train = DatasetCatalog.get("TACO_train")

register_coco_instances("TACO_val", {}, val_annotation_file, img_dir)
MetadataCatalog.get("TACO_val").set(thing_classes = classes)
dataset_dicts_val = DatasetCatalog.get("TACO_val")

[05/17 12:09:34 d2.data.datasets.coco]: Loaded 1200 images in COCO format from /content/TACO-expl/data/annotations_off_0_train.json
[05/17 12:09:34 d2.data.datasets.coco]: Loaded 150 images in COCO format from /content/TACO-expl/data/annotations_off_0_val.json


In [15]:
import json

with open(val_annotation_file, "r") as f:
  val_annotations = json.load(f)

print(val_annotations['annotations'][0])

{'id': 47, 'image_id': 15, 'category_id': 1, 'segmentation': [[249.0, 1458.0, 236.0, 1360.0, 222.0, 1247.0, 150.0, 784.0, 150.0, 747.0, 157.0, 712.0, 172.0, 675.0, 184.0, 656.0, 205.0, 632.0, 238.0, 603.0, 263.0, 589.0, 299.0, 575.0, 312.0, 570.0, 322.0, 568.0, 324.0, 567.0, 323.0, 538.0, 319.0, 497.0, 314.0, 432.0, 307.0, 362.0, 307.0, 354.0, 313.0, 346.0, 317.0, 343.0, 328.0, 338.0, 345.0, 333.0, 355.0, 330.0, 363.0, 328.0, 371.0, 334.0, 386.0, 332.0, 404.0, 330.0, 421.0, 330.0, 450.0, 332.0, 476.0, 335.0, 486.0, 338.0, 489.0, 337.0, 493.0, 335.0, 501.0, 337.0, 509.0, 341.0, 518.0, 346.0, 527.0, 351.0, 531.0, 359.0, 531.0, 369.0, 532.0, 400.0, 533.0, 429.0, 538.0, 526.0, 541.0, 566.0, 545.0, 568.0, 582.0, 580.0, 626.0, 604.0, 655.0, 627.0, 688.0, 667.0, 707.0, 704.0, 717.0, 753.0, 718.0, 806.0, 712.0, 952.0, 706.0, 1090.0, 698.0, 1222.0, 693.0, 1386.0, 688.0, 1433.0, 685.0, 1454.0, 671.0, 1493.0, 644.0, 1539.0, 603.0, 1582.0, 552.0, 1614.0, 498.0, 1630.0, 445.0, 1632.0, 381.0, 1617.0

In [53]:
import json

with open(test_annotation_file, "r") as f:
  test_annotations = json.load(f)

In [ ]:
first_img = val_annotations["images"][0]
first_ann = val_annotations["annotations"][0]

print(first_img)
print(first_ann)

{'id': 899, 'width': 3264, 'height': 2448, 'file_name': 'batch_4/000023.JPG', 'license': None, 'flickr_url': 'https://farm66.staticflickr.com/65535/47803892332_5218b74150_o.png', 'coco_url': None, 'date_captured': None, 'flickr_640_url': 'https://farm66.staticflickr.com/65535/47803892332_f552fb65c7_z.jpg'}
{'id': 47, 'image_id': 15, 'category_id': 1, 'segmentation': [[249.0, 1458.0, 236.0, 1360.0, 222.0, 1247.0, 150.0, 784.0, 150.0, 747.0, 157.0, 712.0, 172.0, 675.0, 184.0, 656.0, 205.0, 632.0, 238.0, 603.0, 263.0, 589.0, 299.0, 575.0, 312.0, 570.0, 322.0, 568.0, 324.0, 567.0, 323.0, 538.0, 319.0, 497.0, 314.0, 432.0, 307.0, 362.0, 307.0, 354.0, 313.0, 346.0, 317.0, 343.0, 328.0, 338.0, 345.0, 333.0, 355.0, 330.0, 363.0, 328.0, 371.0, 334.0, 386.0, 332.0, 404.0, 330.0, 421.0, 330.0, 450.0, 332.0, 476.0, 335.0, 486.0, 338.0, 489.0, 337.0, 493.0, 335.0, 501.0, 337.0, 509.0, 341.0, 518.0, 346.0, 527.0, 351.0, 531.0, 359.0, 531.0, 369.0, 532.0, 400.0, 533.0, 429.0, 538.0, 526.0, 541.0, 566

In [ ]:
!mkdir /content/resized_validation/

In [16]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import itertools

def visualize_mask(mask):
    plt.figure(figsize=(8, 8))
    plt.imshow(mask)
    #plt.axis('off')
    plt.show()

def flatten(aList):
  t = []
  for i in aList:
    if not isinstance(i, list):
      t.append(i)
    else:
      t.extend(flatten(i))
  return t

def mask_to_polygon(mask):
    # Find contours
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    polygons = []
    for contour in contours:
        # Convert polygon to list of coordinates
        polygon = contour.squeeze().astype(float).tolist()
        polygons.append(polygon)
    #polygons = flatten(polygons)
    return polygons

def visualize_polygons(polygons):
    plt.figure(figsize=(8, 8))
    for polygon in polygons:
        polygon = np.array(polygon)
        plt.plot(polygon[:, 0], polygon[:, 1], color='red', linewidth=2)
    plt.axis('off')
    plt.show()

"""
def group_coords_poly(ungrouped_poly):
  to_return = []
  for inner_list in ungrouped_poly:
    to_append = [[inner_list[i], inner_list[i+1]] for i in range(0, len(inner_list), 2)]
    to_return.append(to_append)
  return to_return
"""
def group_coords_poly(ungrouped_poly):
  ung_poly = ungrouped_poly.copy()
  ung_poly = flatten(ung_poly)
  to_return = [[[ung_poly[i], ung_poly[i+1]] for i in range(0, len(ung_poly), 2)]]
  return to_return

def ungroup_coords_poly(grouped_poly):
  #to_return = [[elem for pair_inner_list in inner_list for elem in pair_inner_list] for inner_list in grouped_poly]
  to_return = [flatten(grouped_poly)]
  return to_return

#some_ann = [[2152.0, 1374.0, 2206.0, 1375.0, 2239.0, 1366.0], [2262.0, 1358.0, 2275.0, 1342.0, 2270.0, 1323.0, 2259.0, 1312.0, 2249.0, 1308.0, 2251.0, 1291.0, 2236.0, 1270.0, 2218.0, 1259.0, 2191.0, 1254.0, 2149.0, 1256.0, 2125.0, 1259.0, 2109.0, 1272.0, 2098.0, 1285.0, 2093.0, 1294.0, 2090.0, 1307.0, 2080.0, 1314.0, 2069.0, 1324.0, 2073.0, 1347.0, 2095.0, 1363.0, 2122.0, 1370.0, 2152.0, 1374.0]]
#to_print = group_coords_poly(some_ann)
#print(to_print)
#print(ungroup_coords_poly(to_print))

In [45]:
!mkdir /content/res_val

In [17]:
import numpy as np
from itertools import groupby

def binary_mask_to_rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(groupby(binary_mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    return rle

In [18]:
from detectron2.data.transforms.augmentation_impl import ResizeShortestEdge
from detectron2.data.transforms import ResizeTransform
from PIL import Image
from pycocotools import mask as coco_mask
from skimage import measure

def check_rotation_and_alpha(image):
        img_shape = np.shape(image)

        # load metadata
        exif = image.getexif()
        if exif:
            exif = dict(exif.items())
            # Rotate portrait images if necessary (274 is the orientation tag code)
            if 274 in exif:
                if exif[274] == 3:
                    image = image.rotate(180, expand=True)
                if exif[274] == 6:
                    image = image.rotate(270, expand=True)
                if exif[274] == 8:
                    image = image.rotate(90, expand=True)

        # If has an alpha channel, remove it for consistency
        if img_shape[-1] == 4:
            image = image[..., :3]
        return image



In [48]:
def resize_split(ann_json_string, new_json_path, new_img_folder ):
  resized_json = {}
  resized_json['info'] = ann_json_string['info']
  resized_json['scene_annotations'] = ann_json_string['scene_annotations']
  resized_json['licenses'] = ann_json_string['licenses']
  resized_json['categories'] = ann_json_string['categories']
  resized_json['scene_categories'] = ann_json_string['scene_categories']
  resize_obj_func = ResizeShortestEdge(800,
                                  1333,
                                  sample_style = "choice")
  if not os.path.exists(os.path.join("/content/", new_img_folder)):
    os.mkdir(os.path.join("/content/", new_img_folder))
  res_imgs_json_path = new_json_path#'/content/TACO-expl/data/annotations_off_0_resval.json'

  res_imgs_list = []
  res_annotation_list = []
  for img_meta in ann_json_string["images"]:

    res_img_dict = img_meta.copy()
    id = img_meta["id"]
    height = img_meta["height"]
    width = img_meta["width"]
    print(height, width)
    img_path = img_meta["file_name"]
    sub_dir = img_path.split("/")[0]
    folder = os.path.join("/content/", new_img_folder, sub_dir)
    if not os.path.exists(folder):
      os.mkdir(folder)

    new_h, new_w = ResizeShortestEdge.get_output_shape(height, width, 800, 1333)
    res_img_dict.update({"height": height, "width": width})
    res_imgs_list.append(res_img_dict)

    trans_func = ResizeTransform(height, width, new_h, new_w, Image.BILINEAR)

    img = Image.open(os.path.join("/content/MyDrive/MyDrive/official", img_path))
    img = check_rotation_and_alpha(img)
    img = np.array(img)
    print(img.shape)
    res_img = trans_func.apply_image(img)
    res_pil = Image.fromarray(res_img.astype(np.uint8))
    res_pil.save(os.path.join(folder, img_path.split("/")[1]))

    for annotation in ann_json_string["annotations"]:
      if annotation["image_id"] == id:
        res_segm_dict = annotation.copy()
        rles = coco_mask.frPyObjects(annotation["segmentation"], height, width)
        rle = coco_mask.merge(rles)
        mask = coco_mask.decode(rle)

        res_mask = trans_func.apply_segmentation(mask)
        res_mask = np.asfortranarray(np.squeeze(res_mask))
        uncomp = binary_mask_to_rle(res_mask)


        encoded_ground_truth = coco_mask.encode(res_mask)
        ground_truth_area = float(coco_mask.area(encoded_ground_truth))
        ground_truth_bounding_box = coco_mask.toBbox(encoded_ground_truth).tolist()

        res_segm_dict.update({"segmentation": uncomp,
                            "area": ground_truth_area,
                            "bbox": ground_truth_bounding_box})
        res_annotation_list.append(res_segm_dict)

  resized_json["images"] = res_imgs_list
  resized_json["annotations"] = res_annotation_list
  #with open(res_imgs_json_path, "w") as f:
  #  json.dump(resized_json, f)
  return resized_json


In [31]:
for i in range(len(jso["annotations"])):
  jso["annotations"][i]['area'] = float(jso["annotations"][i]['area'])
  jso["annotations"][i]['bbox'] = jso["annotations"][i]['bbox'].astype('float32')

In [ ]:
jso = resize_split(val_annotations, "/content/TACO-expl/data/annotations_off_0_resval", "res_val" )

In [54]:
jso = resize_split(test_annotations, "/content/TACO-expl/data/annotations_off_0_restest", "res_test" )

4160 3120
(4160, 3120, 3)
4032 3024
(4032, 3024, 3)
2448 3264
(2448, 3264, 3)
4032 3024
(4032, 3024, 3)
1968 2624
(1968, 2624, 3)
2049 1537
(2049, 1537, 3)
3264 2448
(3264, 2448, 3)
2448 3264
(2448, 3264, 3)
3264 2448
(3264, 2448, 3)
4618 3463
(4618, 3463, 3)
2448 3264
(2448, 3264, 3)
4160 3120
(4160, 3120, 3)
3264 2448
(3264, 2448, 3)
5312 2988
(5312, 2988, 3)
4000 1824
(4000, 1824, 3)
3264 2448
(3264, 2448, 3)
2448 3264
(2448, 3264, 3)
4000 6000
(4000, 6000, 3)
5312 2988
(5312, 2988, 3)
4000 1824
(4000, 1824, 3)
3120 4160
(3120, 4160, 3)
4128 2322
(4128, 2322, 3)
2048 1536
(2048, 1536, 3)
2448 3264
(2448, 3264, 3)
3120 4160
(3120, 4160, 3)
3264 2448
(3264, 2448, 3)
2448 3264
(2448, 3264, 3)
3264 2448
(3264, 2448, 3)
4000 1824
(4000, 1824, 3)
3264 2448
(3264, 2448, 3)
4032 3024
(4032, 3024, 3)
4000 1824
(4000, 1824, 3)
3264 2448
(3264, 2448, 3)
2448 3264
(2448, 3264, 3)
5312 2988
(5312, 2988, 3)
2448 3264
(2448, 3264, 3)
2448 3264
(2448, 3264, 3)
1968 2624
(1968, 2624, 3)
4208 2368
(4

In [55]:
jso['annotations'][0]

{'id': 1641,
 'image_id': 464,
 'category_id': 7,
 'segmentation': {'counts': [88022,
   14,
   1051,
   31,
   1033,
   35,
   1030,
   39,
   1026,
   42,
   1023,
   45,
   1019,
   49,
   1016,
   53,
   1012,
   56,
   1010,
   58,
   1008,
   60,
   1006,
   62,
   1005,
   63,
   1003,
   66,
   1000,
   68,
   999,
   69,
   997,
   72,
   994,
   74,
   992,
   76,
   991,
   77,
   989,
   79,
   987,
   81,
   986,
   83,
   983,
   85,
   981,
   87,
   980,
   89,
   977,
   91,
   975,
   93,
   974,
   93,
   973,
   95,
   971,
   97,
   969,
   99,
   967,
   100,
   966,
   102,
   964,
   104,
   962,
   106,
   960,
   107,
   958,
   110,
   957,
   110,
   955,
   113,
   953,
   115,
   951,
   117,
   949,
   118,
   948,
   120,
   945,
   123,
   943,
   124,
   941,
   127,
   938,
   130,
   936,
   131,
   935,
   133,
   932,
   136,
   930,
   138,
   928,
   141,
   924,
   145,
   921,
   148,
   917,
   153,
   913,
   156,
   910,
   160,
   906,
   1

In [56]:
with open("/content/TACO-expl/data/annotations_off_0_restest.json", "w") as f:
    json.dump(jso, f)

In [57]:
!zip -r /content/test.zip /content/res_test/

  adding: content/res_test/ (stored 0%)
  adding: content/res_test/batch_10/ (stored 0%)
  adding: content/res_test/batch_10/000023.jpg (deflated 0%)
  adding: content/res_test/batch_10/000095.jpg (deflated 0%)
  adding: content/res_test/batch_10/000000.jpg (deflated 0%)
  adding: content/res_test/batch_10/000014.jpg (deflated 0%)
  adding: content/res_test/batch_10/000040.jpg (deflated 0%)
  adding: content/res_test/batch_10/000074.jpg (deflated 0%)
  adding: content/res_test/batch_10/000091.jpg (deflated 0%)
  adding: content/res_test/batch_10/000067.jpg (deflated 0%)
  adding: content/res_test/batch_10/000022.jpg (deflated 0%)
  adding: content/res_test/batch_10/000010.jpg (deflated 0%)
  adding: content/res_test/batch_10/000083.jpg (deflated 0%)
  adding: content/res_test/batch_10/000006.jpg (deflated 0%)
  adding: content/res_test/batch_10/000062.jpg (deflated 0%)
  adding: content/res_test/batch_5/ (stored 0%)
  adding: content/res_test/batch_5/000024.JPG (deflated 0%)
  adding: 

In [58]:
!ls

build  functions  modules				  setup.py  test.py
dist   make.sh	  MultiScaleDeformableAttention.egg-info  src
